In [1]:
import sys
sys.path.append('./libs')
import util_modelo_final
import importlib

importlib.reload(util_modelo_final)

tuned_parameters = [{'keras__batch_size': [2**10], 'keras__epochs': [2**10], 'keras__optimizer': ['SGD', 'Adam'], 'keras__learn_rate': [0.01], 'keras__momentum': [0], 'keras__init_mode': ['uniform'], 'keras__activation': ['relu'], 'keras__activation2': ['softmax'], 'keras__dropout_rate': [0.0], 'keras__weight_constraint': [1], 'keras__neurons': [10, 100]}]
tesis2 = util_modelo_final.Tesis2(carpeta_base="prueba",verbose=100,tuned_parameters=tuned_parameters)

cantidad_especies = 30
cantidad_especies = 3
cantidad_transcritos = 4028
cantidad_transcritos = 400

Using TensorFlow backend.


In [2]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, dump

def obtener_especies():
    query = "SELECT * FROM (SELECT CONVERT(@row_number:=@row_number + 1, UNSIGNED) AS orden, m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie, (SELECT @row_number:=0) AS rn ORDER BY m.id_especie) a ORDER BY 1 LIMIT " + str(cantidad_especies)
    return util_bd.resultados_query(query)
        
def generar_fasta_especie(tesis2, row_especie):
    os.mkdir(tesis2.carpeta_data() + "/clase_" + str(row_especie[0]))
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, tesis2.carpeta_data() + "/clase_" + str(row_especie[0]) + "/lncRNA.fa")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, tesis2.carpeta_data() + "/clase_" + str(row_especie[0]) + "/PCT.fa")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + str(row_especie[1]) + " ORDER BY f.cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, tesis2.carpeta_data() + "/clase_" + str(row_especie[0]) + "/CDS.fa")

if os.path.isdir(tesis2.carpeta_data()):
    shutil.rmtree(tesis2.carpeta_data())
os.mkdir(tesis2.carpeta_data())

%time dump(obtener_especies(), tesis2.carpeta_data() + "/info_clases.bin")
%time Parallel(n_jobs=tesis2.n_jobs, verbose=tesis2.verbose)(delayed(generar_fasta_especie)(tesis2, row_especie) for row_especie in obtener_especies())

print("Proceso finalizado")

CPU times: user 2.61 ms, sys: 0 ns, total: 2.61 ms
Wall time: 3.5 ms
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.9s finished
CPU times: user 37.5 ms, sys: 56.2 ms, total: 93.7 ms
Wall time: 5.94 s
Proceso finalizado


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

%time tesis2.generar_modelo_final_keras()

*************************************************
*************** Generando llaves ****************
*************************************************
*************************************************
***************** Armando folds *****************
*************************************************
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1511s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    3.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    3.3s finished
*************************************

{'mean_fit_time': array([10.84318805, 12.95046401, 22.11659193, 21.3222789 ]),
 'std_fit_time': array([0.36528195, 0.79727605, 0.0174501 , 1.43827311]),
 'mean_score_time': array([0.34813944, 0.34410898, 0.42343076, 0.23102029]),
 'std_score_time': array([0.052558  , 0.06679051, 0.09070952, 0.05052675]),
 'param_keras__activation': masked_array(data=['relu', 'relu', 'relu', 'relu'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_keras__activation2': masked_array(data=['softmax', 'softmax', 'softmax', 'softmax'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_keras__batch_size': masked_array(data=[1024, 1024, 1024, 1024],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_keras__dropout_rate': masked_array(data=[0.0, 0.0, 0.0, 0.0],
              mask=[False, False, False, False],
        fill_value='?',
   

{'keras__activation': 'relu',
 'keras__activation2': 'softmax',
 'keras__batch_size': 1024,
 'keras__dropout_rate': 0.0,
 'keras__epochs': 1024,
 'keras__init_mode': 'uniform',
 'keras__learn_rate': 0.01,
 'keras__momentum': 0,
 'keras__neurons': 10,
 'keras__optimizer': 'Adam',
 'keras__weight_constraint': 1}

{'accuracy': 0.8054166666666667,
 'precision': 0.8528194331441183,
 'recall': 0.745}

CPU times: user 12.7 s, sys: 747 ms, total: 13.4 s
Wall time: 2min 30s
